# T5 model

In [1]:
#Load our csv file
import pandas as pd
deval = pd.read_csv("/kaggle/input/deval4/deval4.csv")
pd.options.display.max_colwidth = 300
print(deval.shape)
deval.head(3)

(364, 10)


,IDV,VNAME,total,SUM,t5,rt5,f4t5,rf4t5,pt5,rpt5
0,1,The OVO Hydro,ovo hydro paramore gig apart pavements around hydro great venue found parking spot allows us make use drop kerbs around venue drive venue local subway accessible park round near science centre roll venue 10 minute roll accessible seating area ground floor gives good view stage everyone standings...,"The OVO Hydro is a great venue with good accessibility, including parking, seating, and toilets for disabled patrons.",despite hit redial 2 hours get tickets unless luckier get quicker venue super accessible manage get tickets actually accessing show everything smooth dropped,0.07,ovo hydro sse hydro great place to watch bands ovo hydro sse hydro great place to watch bands,0.21,"The text describes the accessibility features of a venue, including parking, toilets, and helpful staff.",0.50
1,2,Glasgow Royal Concert Hall,glasgow royal concert hall friendly staff ticket centre seeing friendly staff ticket centre staff relatives cousin box office manger communications great got bus difficult access loads stairs lift ramp good signage toilets spotless staff good giving directions seating plan user friendly great ve...,"The Glasgow Royal Concert Hall is a wheelchair-accessible venue with friendly staff, good signage, and spotless toilets.",glasgow royal concert hall friendly staff ticket centre ticket office manger incredibly helpful throughout helped make amazing experience despite problems beginning seats allocated side,0.43,glasgow royal concert hall staff friendly staff ticket centre staff relatives cousin box office manager communications great got bus difficult access loads stairs lift ramp good,0.50,"Glasgow Royal Concert Hall is a wheelchair accessible venue with friendly staff, accessible toilets, and helpful staff.",0.64
2,3,Emirates Arena and Sir Chris Hoy Velodrome,emirates arena sir chris hoy velodrome sports centre glasgow eastend lovely sports centre glasgow plenty disabled parking needed near venue nextbike hire bike docking station centre plenty space personal bikes also easy access venue automatic doors push button access accessible toilets ground fl...,"The Emirates Arena Sir Chris Hoy Velodrome in Glasgow is a highly accessible sports center with plenty of disabled parking, push button access, accessible toilets, and friendly staff.",emirates arena sir chris hoy velodrome sports centre glasgow eastend lovely sports centre,0.36,emirates arena sir chris hoy velodrome glasgow a new building felt a,0.32,"The Emirates Arena Sir Chris Hoy Velodrome is a beautiful sports centre in Glasgow with plenty of disabled parking, accessible toilets, and helpful staff.",0.73


In [8]:
# Checking Rouge using T5
deval['rt5'].describe()

count    364.000000
mean       0.393956
std        0.194583
min        0.000000
25%        0.250000
50%        0.380000
75%        0.540000
max        0.920000
Name: rt5, dtype: float64

In [2]:
# See GPU info (2 GPUs)
!nvidia-smi

Tue Aug  1 22:55:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
#Test Flan-t5 model
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("/kaggle/input/flan-t5/pytorch/base/4")
model = T5ForConditionalGeneration.from_pretrained("/kaggle/input/flan-t5/pytorch/base/4", device_map="auto", torch_dtype=torch.float16)

def get_f4t5(seq): 
    input_ids = tokenizer("summarize: " + seq, return_tensors="pt",
                         max_length=1024, truncation=True).input_ids.to("cuda")

    outputs = model.generate(input_ids, max_length=30, min_length=20
                             , length_penalty=5., num_beams=2)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
#print(get_t5(text))

In [19]:
deval.loc[:, 'f4t5'] = deval.loc[:, 'total'].apply(lambda x: get_f4t5(x))

# Evaluate using Rouge

In [17]:
!pip install Rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from rouge import Rouge
# Initialize the ROUGE object
rouge = Rouge()
stop_words = stopwords.words('english')
def out_stop_words(review):
  tokens = nltk.word_tokenize(review)
  review = [word for word in tokens if word not in stop_words]
  review = ' '.join(review)
  return review.lower()
def get_rouge(g, r):
  g = out_stop_words(g)
  r = out_stop_words(r)
  # Calculate ROUGE for the generated and reference summaries
  scores = rouge.get_scores(g, r)
  # Print the results
  # ['rouge-1']['r']
  return round(scores[0]['rouge-1']['r'], 2)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
for index, row in deval.iterrows():
  deval.loc[index, 'rf4t5'] = get_rouge(deval.loc[index, 'f4t5'], deval.loc[index, 'SUM'])

In [21]:
# Rouge for flan-t5
deval['rf4t5'].describe()

count    364.000000
mean       0.258571
std        0.188503
min        0.000000
25%        0.100000
50%        0.230000
75%        0.380000
max        0.850000
Name: rf4t5, dtype: float64

In [34]:
deval.to_csv("/kaggle/working/deval3.csv",index=False)

In [23]:
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
trace1=go.Box(x=deval['rt5'].values , boxpoints='all')
trace2=go.Box(x=deval['rf4t5'].values , boxpoints='all')
figure=go.Figure(data=[trace1, trace2])
figure.update_layout(
    title = "Rouge generate v reference"
)
figure.update_traces(orientation='h')
figure.show()

# Fine tune 1

In [2]:
deval = deval.rename(columns={"SUM":"target_text", "total":"source_text"})
deval = deval[['source_text', 'target_text']]
deval['source_text'] = "summarize: " + deval['source_text']
deval.head(2)

,source_text,target_text
0,summarize: ovo hydro paramore gig apart pavements around hydro great venue found parking spot allows us make use drop kerbs around venue drive venue local subway accessible park round near science centre roll venue 10 minute roll accessible seating area ground floor gives good view stage everyon...,"The OVO Hydro is a great venue with good accessibility, including parking, seating, and toilets for disabled patrons."
1,summarize: glasgow royal concert hall friendly staff ticket centre seeing friendly staff ticket centre staff relatives cousin box office manger communications great got bus difficult access loads stairs lift ramp good signage toilets spotless staff good giving directions seating plan user friend...,"The Glasgow Royal Concert Hall is a wheelchair-accessible venue with friendly staff, good signage, and spotless toilets."


In [3]:
# Split train and test to fine tune our t5
!pip install simplet5
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5
train_df, test_df = train_test_split(deval, test_size=0.2, random_state=42)
print("Size to fine tune", train_df.shape, test_df.shape)

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 38.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 55.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=515d889983a63f40872e67dc2f0da0b7d0b4160f6dc64b5405caf6879621d6a2
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3a1638d9a6242b1143f35dc9c000e83241f513523b17cdfba015bb2ea7d4c54f
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built simplet5 sacremoses
  Attempting uninsta

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Size to fine tune (291, 2) (73, 2)


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [5]:
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=512, 
            target_max_token_len=64, 
            batch_size=8, 
            max_epochs=3, 
            use_gpu=True
           )

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [5]:
# to download the model
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
download_file('/kaggle/working/outputs/simplet5-epoch-3-train-loss-1.0959-val-loss-1.4198', 'out')

/kaggle/working/out.zip

In [9]:
# Load the fune tuned model
model.load_model("t5","/kaggle/working/outputs/simplet5-epoch-2-train-loss-1.1119-val-loss-1.3865", use_gpu=True)

In [15]:
# Test in one cell
x = model.predict("summarize: " + deval.loc[2, 'total'])
print(x[0])
# print(deval.loc[1, 'target_text'])

The Emirates Arena Sir Chris Hoy Velodrome is a beautiful sports centre in Glasgow with plenty of disabled parking, accessible toilets, and helpful staff.


In [16]:
deval.loc[:, 'pt5'] = deval.loc[:, 'total'].apply(lambda x: model.predict("summarize: " + x)[0])

In [19]:
for index, row in deval.iterrows():
  deval.loc[index, 'rpt5'] = get_rouge(deval.loc[index, 'pt5'], deval.loc[index, 'SUM'])

In [20]:
deval['rpt5'].describe()

count    364.000000
mean       0.619011
std        0.176458
min        0.220000
25%        0.500000
50%        0.620000
75%        0.750000
max        1.000000
Name: rpt5, dtype: float64

In [21]:
deval.to_csv("/kaggle/working/deval4.csv",index=False)

In [5]:
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
trace1=go.Box(x=deval['rt5'].values , boxpoints='all')
trace2=go.Box(x=deval['rf4t5'].values , boxpoints='all')
trace3=go.Box(x=deval['rpt5'].values , boxpoints='all')
figure=go.Figure(data=[trace1, trace2, trace3])
figure.update_layout(
    title = "Rouge generate v reference"
)
figure.update_traces(orientation='h')
figure.show()

In [6]:
# Histogram
fig = px.histogram(deval, x="rpt5", nbins=20)
fig.update_layout(
    title = "Rouge fine tuned T5 generate v reference"
)
fig.show()

# Fine tune New Attempt

In [4]:
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=768, 
            target_max_token_len=40, 
            batch_size=4, 
            max_epochs=4, 
            use_gpu=True
           )

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]